# TREINAMENTO ML

In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np
import os
import warnings
import seaborn as sns
import matplotlib.pyplot as plt

pd.options.display.float_format = '{:.2f}'.format
warnings.filterwarnings("ignore")

# url_hourly = "https://media.githubusercontent.com/media/ruanvirginio/masters/refs/heads/main/bases_tratadas/transformers_dataset.csv"
# df_hourly = pd.read_csv(url_hourly,  sep=';', encoding='latin-1')

url_daily = "https://media.githubusercontent.com/media/ruanvirginio/masters/refs/heads/main/bases_tratadas/daily_peak_transformers_dataset.csv"
df_daily = pd.read_csv(url_daily,  sep=';', encoding='latin-1')

df = df_daily

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'vscode'

from math import sqrt
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import joblib

from sklearn.base import clone  

# keras_tuner para otimização das redes (usada uma vez por trafo/modelo)
import tensorflow as tf
tf.config.threading.set_intra_op_parallelism_threads(16)
tf.config.threading.set_inter_op_parallelism_threads(16)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.callbacks import EarlyStopping
import keras_tuner as kt

import matplotlib
matplotlib.use('Agg')  # & para evitar backend interativo em servidores

np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

# ========================================================================
# FUNÇÕES AUXILIARES (PLOTS, MÉTRICAS, EXPORTAÇÃO)
# ========================================================================
def gerar_tabela_metricas_por_fold(trafo, modelo, fold_rmse, fold_mae):
    # Gera um DataFrame com RMSE/MAE de cada fold para exibir e salvar
    return pd.DataFrame({
        'Fold': [f'Fold {i+1}' for i in range(len(fold_rmse))],
        'Trafo': trafo,
        'Modelo': modelo,
        'RMSE': np.round(fold_rmse, 4),
        'MAE': np.round(fold_mae, 4)
    })

def plotar_ultimo_fold(datas, y_real, y_pred, trafo, modelo, freq):
    plt.figure(figsize=(14,6))
    plt.plot(datas, y_real, label='Real', color='blue')
    plt.plot(datas, y_pred, label=f'Predicted ({modelo})', linestyle='--', color='orange')
    plt.xlabel('Day' if freq=='daily' else 'Hour')
    plt.ylabel('Apparent Power (kVA)')
    plt.title(f'Last Fold Prediction - {modelo} ({trafo})')
    plt.legend()
    plt.tight_layout()
    os.makedirs('plots', exist_ok=True)
    plt.savefig(f'plots/PLOT_{modelo}_{trafo}_{freq}_ultimo_fold.pdf', dpi=300, bbox_inches='tight')
    plt.close()

def plotar_todos_folds(lista_datas, lista_reais, lista_previstos, trafo, modelo, eixo_label='Data'):
    # Plota todos os folds concatenados em uma figura interativa (Plotly)
    if len(lista_datas) == 0:
        return
    datas_todas = pd.to_datetime(np.concatenate(lista_datas))
    reais_todos = np.concatenate(lista_reais)
    previstos_todos = np.concatenate(lista_previstos)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=datas_todas, y=reais_todos, mode='lines', name='Real'))
    fig.add_trace(go.Scatter(x=datas_todas, y=previstos_todos, mode='lines', name=f'Previsto ({modelo})', line=dict(dash='dash')))
    fig.update_layout(title=f'Previsão em Todos os Folds - {trafo} ({modelo})',
                      xaxis_title=eixo_label, yaxis_title='Potência Aparente', hovermode='x unified')
    fig.show()

# ========================================================================
# GRIDS BASE PARA HYPERPARAMS
# ========================================================================
param_grids_base = {
    'SVR': {
        'C': [0.1, 1, 10, 100],
        'gamma': ['scale', 'auto', 0.01, 0.1],
        'epsilon': [0.01, 0.1, 0.5]
    },
    'RFR': {
        'n_estimators': [50, 100, 150],
        'max_depth': [5, 10, 15],
        'min_samples_split': [2, 5]
    },
    'GBR': {
        'n_estimators': [50, 100, 150],
        'learning_rate': [0.05, 0.1],
        'max_depth': [3, 5]
    },
    'XGB': {
        'n_estimators': [50, 100, 150],
        'learning_rate': [0.05, 0.1],
        'max_depth': [3, 5]
    },
    'LGBM': {
        'n_estimators': [50, 100, 150],
        'learning_rate': [0.05, 0.1],
        'max_depth': [5, 7]
    }
}

def get_param_grids_por_freq(freq, modelo):
    # Ajusta o grid padrão de acordo com a granularidade dos dados, com reduções para dados horários
    base_params = param_grids_base[modelo].copy()
    if freq == 'hourly':
        if modelo == 'SVR':
            base_params['C'] = [1, 10]
            base_params['gamma'] = ['scale', 0.01]
            base_params['epsilon'] = [0.1]
        elif modelo in ['RFR', 'GBR', 'XGB', 'LGBM']:
            base_params['n_estimators'] = [50, 100]
    return base_params

# ========================================================================
# FUNÇÃO AUXILIAR: CRIA MODELO KERAS A PARTIR DE HYPERPARAMS FIXOS
# ========================================================================
def build_keras_from_hps_fixed(modelo, hps):
    # Recebe o nome do modelo (LSTM/CNN/CNN_LSTM) e um dict-like de hyperparams. Retorna um modelo Keras já compilado pronto para treinar
    model = Sequential()
    if modelo == 'LSTM':
        # % Para LSTM pegamos 'units1' (número de neurônios) do best_hps
        model.add(LSTM(units=int(hps.get('units1', 20)), input_shape=(None, 1)))
        model.add(Dense(1))
    elif modelo == 'CNN':
        # % CNN simples 1D: filters e kernel são fixos nos valores do tuner
        model.add(Conv1D(filters=int(hps.get('filters', 32)), kernel_size=2, activation='relu', input_shape=(None, 1)))
        model.add(MaxPooling1D(2))
        model.add(Flatten())
        model.add(Dense(1))
    elif modelo == 'CNN_LSTM':
        # % CNN + LSTM: primeiro convolucional, depois LSTM
        model.add(Conv1D(filters=int(hps.get('filters', 32)), kernel_size=2, activation='relu', input_shape=(None, 1)))
        model.add(MaxPooling1D(2))
        model.add(LSTM(units=int(hps.get('units1', 20))))
        model.add(Dense(1))
    else:
        raise ValueError(f"Modelo Keras desconhecido: {modelo}")

    model.compile(optimizer='adam', loss='mse')
    return model

# ========================================================================
# FUNÇÃO PRINCIPAL: OTIMIZAÇÃO UMA-VEZ + TREINO POR FOLD COM PARAMS FIXOS
# ========================================================================
def treinar_e_prever_modelo_auto_sem_data_leakage(
    data, trafos_escolhidos, modelo, janela=None, epochs=15, batch_size=32, n_iter_search=5
):
    """
    Após detectar frequência desejada, faz para cada trafo:
       1) Prepara janelas (sliding window)
       2) Executa busca de hiperparâmetros no conjunto inicial (tuning set)
       3) Usa os hiperparâmetros encontrados para treinar/avaliar cada fold (TimeSeriesSplit)
       4) Salva modelos, métricas e parâmetros
    """
    resultados = []
    parametros_finais = []
    os.makedirs('modelos', exist_ok=True)
    data = data.copy()
    data['datahora'] = pd.to_datetime(data['datahora'])

    # Detecta granularidade pela mediana do delta entre timestamps
    delta = data['datahora'].diff().median()
    freq = 'daily' if delta >= pd.Timedelta('1D') else 'hourly'
    if janela is None:
        janela = 30 if freq == 'daily' else 24*3

    # Percorre cada trafo solicitado
    for trafo in trafos_escolhidos:
        print(f"\n Treinando {modelo} para {trafo} ({freq})...")

        # Prepara os dados do trafo (ordenados e sem NA)
        df_trafo = data[data['id'] == trafo][['datahora', 'S']].sort_values('datahora').dropna()
        S_values = df_trafo['S'].values
        datas = df_trafo['datahora'].values

        n_samples = len(S_values) - janela
        if n_samples <= 0:
            print(f"Dados insuficientes para {trafo}. Pulando...")
            continue

        # % Cria janelas deslizantes (sliding window). Para X: cada amostra é uma janela de tamanho 'janela'; y é o valor seguinte
        X = np.lib.stride_tricks.sliding_window_view(S_values, janela)[:n_samples]
        y = S_values[janela:janela + n_samples]
        datas_janela = datas[janela:janela + n_samples]

        # % Ajuste de forma para modelos sklearn vs Keras
        if modelo in ['LSTM', 'CNN', 'CNN_LSTM']:
            # shape = (n_samples, janela, 1)
            X_keras = X.reshape((X.shape[0], X.shape[1], 1))
        else:
            # shape = (n_samples, janela)
            X_sklearn = X.reshape((X.shape[0], -1))

        # ============================
        # ETAPA 1: HYPERPARAM SEARCH (1x)
        # ============================
        if modelo in param_grids_base.keys():
            # Modelos scikit-learn: RandomizedSearchCV executado apenas uma vez
            print("Executando RandomizedSearchCV apenas uma única vez (sem leakage)...")
            param_grid_freq = get_param_grids_por_freq(freq, modelo)

            base_model = {
                'SVR': SVR(),
                'RFR': RandomForestRegressor(random_state=42, n_jobs=-1),
                'GBR': GradientBoostingRegressor(random_state=42),
                'XGB': XGBRegressor(random_state=42, objective='reg:squarederror'),
                'LGBM': LGBMRegressor(random_state=42, n_jobs=-1, verbose=-1)
            }[modelo]


            # Define um conjunto inicial (tuning set) para busca de hiperparâmetros. Usar os primeiros 70% dos exemplos evita usar informações do futuro
            split_point = int(0.7 * len(X_sklearn))
            X_tune, y_tune = X_sklearn[:split_point], y[:split_point]

            # RandomizedSearchCV com TimeSeriesSplit interno
            search = RandomizedSearchCV(
                base_model,
                param_distributions=param_grid_freq,
                n_iter=n_iter_search,
                scoring='neg_mean_squared_error',
                cv=TimeSeriesSplit(n_splits=3),
                n_jobs=-1 if modelo != 'SVR' else 1,
                random_state=42,
                verbose=0
            )
            search.fit(X_tune, y_tune)
            best_params = search.best_params_
            best_score = search.best_score_
            print(f"Melhores parâmetros encontrados (tuning): {best_params}")
            # Registro do que foi encontrado
            parametros_finais.append({
                'Trafo': trafo,
                'Modelo': modelo,
                'Frequência': freq,
                'Melhores_Parâmetros': best_params,
                'Melhor_Score_Tuning': best_score,
                'Metodo_Tuning': 'RandomizedSearchCV'
            })

        else:
            # Redes neurais: usamos KerasTuner apenas uma vez no tuning set
            print("Otimizando rede neural apenas uma vez com KerasTuner (sem leakage)...")

            # & Criador de modelo para o tuner (recebe hp)
            def build_nn_tuner(hp):
                model = Sequential()
                if modelo == 'LSTM':
                    units1 = hp.Int('units1', 20, 80, step=20)
                    model.add(LSTM(units=units1, input_shape=(X_keras.shape[1], 1)))
                    model.add(Dense(1))
                elif modelo == 'CNN':
                    filters = hp.Int('filters', 32, 64, step=32)
                    model.add(Conv1D(filters=filters, kernel_size=2, activation='relu', input_shape=(X_keras.shape[1], 1)))
                    model.add(MaxPooling1D(2))
                    model.add(Flatten())
                    model.add(Dense(1))
                elif modelo == 'CNN_LSTM':
                    filters = hp.Int('filters', 32, 64, step=32)
                    units1 = hp.Int('units1', 20, 60, step=20)
                    model.add(Conv1D(filters=filters, kernel_size=2, activation='relu', input_shape=(X_keras.shape[1], 1)))
                    model.add(MaxPooling1D(2))
                    model.add(LSTM(units=units1))
                    model.add(Dense(1))
                model.compile(optimizer='adam', loss='mse')
                return model

            # Prepara tuning set (70% inicial)
            split_point = int(0.7 * len(X_keras))
            X_tune, y_tune = X_keras[:split_point], y[:split_point]

            # Configura o tuner (Random search, poucas trials para economizar tempo)
            tuner = kt.RandomSearch(
                build_nn_tuner,
                objective='val_loss',
                max_trials=5,           # ajuste: 5 para balancear custo/benefício
                executions_per_trial=1,
                overwrite=True,
                directory='keras_tuner',
                project_name=f'{modelo}_{trafo}_{freq}'
            )

            # Early stopping usado na busca para acelerar e evitar overfitting
            early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

            tuner.search(X_tune, y_tune,
                         epochs=max(5, epochs//2),    # usa menos epochs no tuner
                         batch_size=batch_size,
                         validation_split=0.2,
                         callbacks=[early_stop],
                         verbose=0)

            best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
            # Convertendo os hyperparams do tuner para um dict plano para salvar
            best_params = best_hps.values
            parametros_finais.append({
                'Trafo': trafo,
                'Modelo': modelo,
                'Frequência': freq,
                'Melhores_Parâmetros': best_params,
                'Melhor_Score_Tuning': None,
                'Metodo_Tuning': 'KerasTuner'
            })
            print(f"Melhores parâmetros encontrados (tuning Keras): {best_params}")

        # ============================
        # ETAPA 2: AVALIAÇÃO EM TIME-SERIES SPLIT (USANDO PARAMS FIXOS)
        # ============================
        print("Avaliando com TimeSeriesSplit (parâmetros fixos em todos os folds)...")
        tscv = TimeSeriesSplit(n_splits=5)
        fold_rmse, fold_mae = [], []
        lista_datas, lista_reais, lista_previstos = [], [], []

        for fold_idx, (train_idx, test_idx) in enumerate(tscv.split(X if modelo in ['LSTM','CNN','CNN_LSTM'] else X_sklearn)):
            # Usa índices do TimeSeriesSplit para criar conjuntos de treino/teste
            if modelo in ['LSTM','CNN','CNN_LSTM']:
                X_train, X_test = X_keras[train_idx], X_keras[test_idx]
            else:
                X_train, X_test = X_sklearn[train_idx], X_sklearn[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            # Treina com os hiperparâmetros fixos obtidos anteriormente
            if modelo in param_grids_base.keys():
                # Instancia o modelo sklearn com os melhores parâmetros
                estimator = {
                    'SVR': SVR(),
                    'RFR': RandomForestRegressor(random_state=42, n_jobs=-1),
                    'GBR': GradientBoostingRegressor(random_state=42),
                    'XGB': XGBRegressor(random_state=42, objective='reg:squarederror'),
                    'LGBM': LGBMRegressor(random_state=42, n_jobs=-1, verbose=-1)
                }[modelo]

                # Aplica os parâmetros encontrados (best_params)
                estimator.set_params(**best_params)
                estimator.fit(X_train, y_train)
                y_pred = estimator.predict(X_test)

                # Salva o modelo final (substitui para cada fold: se quiser só o final, salvar fora do loop)
                model_path = f"modelos/{modelo}_{trafo}_{freq}.pkl"
                joblib.dump(estimator, model_path)

            else:
                # Para redes neurais, constrói um novo modelo a partir dos HPS fixos e treina por fold
                model = build_keras_from_hps_fixed(modelo, best_params)
                early_stop_fold = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
                model.fit(X_train, y_train,
                          epochs=epochs,
                          batch_size=batch_size,
                          validation_split=0.1,
                          callbacks=[early_stop_fold],
                          verbose=0)
                y_pred = model.predict(X_test).flatten()

                # Salva modelo keras (usamos o último fold como modelo salvo)
                model_path = f"modelos/{modelo}_{trafo}_{freq}.h5"
                model.save(model_path)

            # Calcula métricas do fold e guarda resultados para plots/resumo
            rmse = sqrt(mean_squared_error(y_test, y_pred))
            mae = mean_absolute_error(y_test, y_pred)
            fold_rmse.append(rmse)
            fold_mae.append(mae)
            datas_finais = datas_janela[test_idx]
            lista_datas.append(datas_finais)
            lista_reais.append(y_test)
            lista_previstos.append(y_pred)

            print(f"   Fold {fold_idx+1}: RMSE={rmse:.4f}, MAE={mae:.4f}")

        df_metricas = gerar_tabela_metricas_por_fold(trafo, modelo, fold_rmse, fold_mae)
        print(df_metricas)

        plotar_ultimo_fold(lista_datas[-1], lista_reais[-1], lista_previstos[-1], trafo, modelo, freq)
        plotar_todos_folds(lista_datas, lista_reais, lista_previstos, trafo, modelo, eixo_label='Dia' if freq=='daily' else 'Hora')

        resultados.append({
            'Trafo': trafo,
            'Modelo': modelo,
            'Frequência': freq,
            'RMSE Médio': np.round(np.mean(fold_rmse), 4),
            'MAE Médio': np.round(np.mean(fold_mae), 4),
            'RMSE Último Fold': np.round(fold_rmse[-1], 4),
            'MAE Último Fold': np.round(fold_mae[-1], 4),
            'Modelo_Salvo': model_path
        })

    resultados_df = pd.DataFrame(resultados)
    parametros_df = pd.DataFrame(parametros_finais)
    return resultados_df, parametros_df

# ========================================================================
# FUNÇÕES AUXILIARES DE RELATÓRIO / SALVAMENTO DE PARÂMETROS
# ========================================================================
def gerar_relatorio_parametros(resultados_df, parametros_df):
    # % Gera um relatório simples com os melhores parâmetros (último tuning salvo)
    relatorio_completo = []
    for _, row in resultados_df.iterrows():
        trafo = row['Trafo']
        modelo = row['Modelo']
        freq = row['Frequência']
        params_ = parametros_df[
            (parametros_df['Trafo'] == trafo) &
            (parametros_df['Modelo'] == modelo) &
            (parametros_df['Frequência'] == freq)
        ]
        if not params_.empty:
            best_params = params_.iloc[-1]['Melhores_Parâmetros']
            relatorio_completo.append({
                'Modelo': modelo,
                'Trafo': trafo,
                'Frequência': freq,
                'RMSE_Médio': row['RMSE Médio'],
                'MAE_Médio': row['MAE Médio'],
                'Parâmetros_Otimizados': best_params
            })
    return pd.DataFrame(relatorio_completo)

def salvar_parametros_detalhados(parametros_df, nome_arquivo='parametros_hiperparametrizacao.csv'):
    # % Expande dicionário de hyperparams em colunas e salva CSV
    registros = []
    for _, row in parametros_df.iterrows():
        base = {
            'Trafo': row.get('Trafo'),
            'Modelo': row.get('Modelo'),
            'Frequência': row.get('Frequência'),
            'Metodo_Tuning': row.get('Metodo_Tuning'),
            'Melhor_Score_Tuning': row.get('Melhor_Score_Tuning')
        }
        best = row.get('Melhores_Parâmetros', {})
        if isinstance(best, dict):
            for k, v in best.items():
                base[f'Param_{k}'] = v
        else:
            # % Quando o tuner retorna um dict-like com tipos diferentes
            try:
                for k, v in dict(best).items():
                    base[f'Param_{k}'] = v
            except Exception:
                base['Param_raw'] = str(best)
        registros.append(base)
    df_expandido = pd.DataFrame(registros)
    os.makedirs('resultados', exist_ok=True)
    caminho = os.path.join('resultados', nome_arquivo)
    df_expandido.to_csv(caminho, index=False, encoding='utf-8')
    print(f"Parâmetros detalhados salvos em: {caminho}")
    return df_expandido

# ========================================================================
# ========================================================================
# ========================================================================

trafos = ['T21a', 'T40', 'T66', 'T4', 'T6', 'T10', 'T51', 'T57', 'T71']
modelos_a_treinar = ['SVR','RFR','GBR','LGBM','LSTM']
resultados_geral = []
parametros_geral = []

for m in modelos_a_treinar:
    res_df, param_df = treinar_e_prever_modelo_auto_sem_data_leakage(df_daily, trafos, m, janela=30, epochs=15, batch_size=32, n_iter_search=5)
    resultados_geral.append(res_df)
    parametros_geral.append(param_df)
resultados_geral_df = pd.concat(resultados_geral, ignore_index=True)
parametros_geral_df = pd.concat(parametros_geral, ignore_index=True)
salvar_parametros_detalhados(parametros_geral_df, nome_arquivo='parametros_todos_modelos.csv')
